In [1]:
import pandas as pd

In [2]:
featuresData = pd.\
    read_csv("../dataset/feature_regression_example.csv")

In [3]:
featuresData.head(2)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442


In [4]:
import pandas as pd

In [5]:
import numpy as np

In [6]:
import numpy as np
import pandas as pd 

# 모델 라이브러리 선언
from sklearn import datasets, tree

# 모델 정확도 라이브러리 선언
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

# CSV 파일을 읽어 DataFrame 변수에 저장하기
featuresData = pd.read_csv("../dataset./feature_regression_example.csv")

In [7]:
featuresData.columns

Index(['REGIONID', 'PRODUCTGROUP', 'PRODUCT', 'ITEM', 'YEARWEEK', 'YEAR',
       'WEEK', 'QTY', 'HOLIDAY', 'HCLUS', 'PROMOTION', 'PRO_PERCENT'],
      dtype='object')

In [8]:
featuresData.PROMOTION.unique()

array(['Y', 'N'], dtype=object)

### 1. np.where

In [9]:
featuresData["PRO_YN"] = \
    np.where(featuresData.PROMOTION == "Y", 1,  0)
featuresData["HO_YN"] = \
    np.where(featuresData.HOLIDAY == "Y", 1,  0)

In [10]:
#### 1. pd.getdummies

In [11]:
pd.concat(  [ featuresData,
               pd.get_dummies( data = featuresData.PROMOTION,
                           prefix = "PROMO")],
           axis = 1).head(2)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PRO_YN,HO_YN,PROMO_N,PROMO_Y
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442,1,1,0,1
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442,1,0,0,1


### 2. Label Encoder

In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
le_pro = LabelEncoder()

In [14]:
featuresData["ENCO_PRO"] = \
    le_pro.fit_transform(featuresData.PROMOTION)

In [15]:
featuresData["INV_PRO"] = \
    le_pro.inverse_transform(featuresData.ENCO_PRO)

### 3. Dictionary

In [16]:
promoDict = { "Y":999,
              "N":100 }

In [17]:
featuresData["DICT_PRO"] = \
        featuresData.PROMOTION.map(promoDict)

In [18]:
cleansedData = featuresData[   (featuresData.YEARWEEK >= 201501) &
                               (featuresData.YEARWEEK <= 201652)   ]

In [19]:
corrdf = cleansedData.corr()

In [20]:
corrStd = 0.5

In [21]:
features = list(corrdf[ (abs(corrdf["QTY"]) > corrStd) & 
         (corrdf["QTY"] < 1)   ].index)
features

['HCLUS', 'PRO_PERCENT', 'PRO_YN', 'HO_YN', 'ENCO_PRO', 'DICT_PRO']

In [22]:
label = ["QTY"]
label

['QTY']

In [23]:
targetColumns = ["YEARWEEK", "YEAR"]

In [24]:
indexStd = round(len(featuresData)*0.7)

In [25]:
featuresData.loc[ indexStd ]["YEARWEEK"]

201623

In [26]:
indexStd = round(len(featuresData)*0.8)

In [27]:
yearweekStd = featuresData.loc[indexStd]["YEARWEEK"]
yearweekStd

201634

In [28]:
yearweekStd = 201630

In [29]:
trainingData_features = \
            featuresData[   featuresData.YEARWEEK <= yearweekStd][  features  ]
trainingData_label = \
            featuresData[   featuresData.YEARWEEK <= yearweekStd][  label  ]
testData_features = \
            featuresData[   featuresData.YEARWEEK > yearweekStd][  features  ]
testData_label = \
            featuresData[   featuresData.YEARWEEK > yearweekStd][  label  ]
testData_all = \
            featuresData[   featuresData.YEARWEEK > yearweekStd]

In [30]:
print(trainingData_features.shape)
print(trainingData_label.shape)
print(testData_features.shape)
print(testData_label.shape)

(83, 6)
(83, 1)
(24, 6)
(24, 1)


In [31]:
# Define Model
model_method = tree.DecisionTreeRegressor(random_state=1)

# Learning (Feature & Label)
model = model_method.fit(trainingData_features, \
                         trainingData_label)

In [32]:
# 예측
predict = model.predict(testData_features)
predict

array([1606.5       , 1606.5       ,  350.71428571, 1434.6       ,
       1434.6       , 1434.6       , 1434.6       , 1434.6       ,
       1606.5       , 1606.5       , 2620.42857143, 1606.5       ,
        350.71428571, 2193.8       , 2193.8       , 2193.8       ,
       2193.8       , 2193.8       , 2193.8       , 2193.8       ,
       2193.8       , 2193.8       ,  350.71428571,  350.71428571])

In [36]:
testData_all["PREDICT"] = predict

C:\Users\kopo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### 결과검증

In [39]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [40]:
mean_absolute_error( testData_all["QTY"],
                     testData_all["PREDICT"])

450.3327380952381

In [41]:
np.sqrt(mean_squared_error( testData_all["QTY"],
                     testData_all["PREDICT"]))

586.3704777283391